In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

#### 超参数  
超参数是可调节的参数，可以让您控制模型优化过程。不同的超参数值会影响模型的训练和收敛速度  
- **Number of Epochs** - the number times to iterate over the dataset  
- **Batch Size** - the number of data samples propagated through the network before the parameters are updated  
- **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

#### 优化循环  
优化循环的每次迭代称为一个epoch  
每个epoch包含两个部分：  
- 训练循环：迭代训练数据集，并尝试收敛到最优参数
- 验证/测试循环：迭代测试数据集以检查模型性能是否得到了改善

#### Loss Function  
Loss function度量得到的结果与目标值的不相似程度，是我们在训练过程中希望最小化的。为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实的数据标签值进行比较。

常用的损失函数包括nn.MSELoss(Mean Square Error, MSELoss), **通常用于回归任务**;nn.NLLLoss(负对数似然)**通常用于分类任务**。nn.CrossEntropyLoss结合了nn.LogSoftmax nn.NLLLoss。

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

#### Optimizer
优化是在每个训练步骤中调整模型参数以减少模型误差的过程。优化算法定义了这个过程是如何执行的(在这个例子中，我们使用随机梯度下降法)。所有优化逻辑都封装在optimizer对象中。

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中优化发生在三个步骤：  
- 调用optimizer.zero_grad()重置模型参数的梯度。梯度默认情况下是叠加的;为了防止重复计算，我们在每次迭代时显式地将它们归零。  
- 通过调用loss.backward()反向传播预测损失。PyTorch保存每个参数关于loss的梯度。  
- 一旦我们有了梯度，我们调用optimizer.step()来通过反向传播中收集的梯度来调整参数。

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 计算预测和损失
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

    
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:2.144979 [    0/60000]
loss:2.123821 [ 6400/60000]
loss:2.062800 [12800/60000]
loss:2.088057 [19200/60000]
loss:2.025103 [25600/60000]
loss:1.965136 [32000/60000]
loss:2.000467 [38400/60000]
loss:1.909543 [44800/60000]
loss:1.918382 [51200/60000]
loss:1.845283 [57600/60000]
Test Error: 
 Accuracy: 53.4%, Avg loss:1.842798 

Epoch 2
-------------------------------
loss:1.874499 [    0/60000]
loss:1.832737 [ 6400/60000]
loss:1.719008 [12800/60000]
loss:1.769803 [19200/60000]
loss:1.656675 [25600/60000]
loss:1.616400 [32000/60000]
loss:1.642071 [38400/60000]
loss:1.551454 [44800/60000]
loss:1.573878 [51200/60000]
loss:1.471884 [57600/60000]
Test Error: 
 Accuracy: 61.1%, Avg loss:1.495325 

Epoch 3
-------------------------------
loss:1.558909 [    0/60000]
loss:1.523204 [ 6400/60000]
loss:1.385178 [12800/60000]
loss:1.456675 [19200/60000]
loss:1.341773 [25600/60000]
loss:1.340666 [32000/60000]
loss:1.352152 [38400/60000]
loss:1.292738 [44800/6